In [4]:
##inspired by https://github.com/benyaminahmed/nft-image-generator/blob/main/generate.ipynb
from PIL import Image 
from IPython.display import display 
import random
import json

TOTAL_IMAGES = 5000

In [31]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%


##list of all trait categories - ordered by layer (background first, then body, ...)
all_traits = ['background', 'body-color','mouth','eyes','clothes','headwear','accessories']

##map all traits with their occurences
with open('constants/attribute-weights.json', 'r') as f:
    all_attributes = json.load(f)

In [25]:
## Generate Traits

all_images = [] 

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} #

    for trait in all_traits:
        new_image[trait] = random.choices(
            [x['name'] for x in all_attributes[trait]],
            [x['weight'] for x in all_attributes[trait]]
        )[0]
    
    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    
    new_trait_image = create_new_image()
    
    all_images.append(new_trait_image)

In [27]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [28]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [30]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [35]:
#### Generate Images    
for item in all_images[:10]:
    
    layers=[]
    
    for trait in all_traits:
        if item[trait]!='None':
            layers.append(
                Image.open(
                    f'./layers/{trait}/{item[trait]}.png'
                ).convert('RGBA')
            )
    
    #create base layer
    com = Image.alpha_composite(layers[0], layers[1])
    
    for i in range(2, len(layers)):
        com = Image.alpha_composite(com, layers[i])
        
    #Convert to RGB
    rgb_im = com.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)